### Вымышленная игра состоит из 100 уровней и собирает следующую статистику о действиях игроков:
- - - -
* user_id - уникальный айди игрока
* completed_lvls - cколько уровней пройдено в момент события
* action_type - какое событие произошло в игре
* Всего возможно 5 событий:
* start – когда игрок нажимает «Начать уровень»
* quit – когда игрок нажимает «Выйти», находясь внутри уровня
* hide – когда игрок сворачивает игру, находясь внутри уровня
* lose – когда игрок проигрывает уровень 
* win – когда игрок побеждает на уровне (после победы он начинает следующий уровень)

### Задание
* Для каждого уровня посчитайте следующие метрики:
* Количество уникальных игроков, начавших уровень хотя бы раз
* Суммарное количество стартов уровня (включая повторные попытки)
* Суммарное количество побед на уровне
* % уникальных игроков, начавших уровень, по отношению к игрокам, начавшим уровень 1
* % уникальных игроков, начавших уровень, по отношению к игрокам, начавшим предыдущий уровень
* Винрейт, т.е. отношение побед на уровне к суммарному количеству стартов уровня


In [1]:
import pandas as pd

In [13]:
df = pd.read_csv (r'/Users/soficodanilyantz/Downloads/Тестовые задания для аналитика (DouDouGames) NEW/levels.csv')
df

,user_id,completed_lvls,action_type
0,ec9d80e3-6cf0-4fa9-800a-6e77bd62409d,0,start
1,ec9d80e3-6cf0-4fa9-800a-6e77bd62409d,1,win
2,ec9d80e3-6cf0-4fa9-800a-6e77bd62409d,1,start
3,ec9d80e3-6cf0-4fa9-800a-6e77bd62409d,2,win
4,ec9d80e3-6cf0-4fa9-800a-6e77bd62409d,2,start
...,...,...,...
104194,36fe3f0f-2d2c-4daf-bffc-7a9e84b82d86,76,win
104195,36fe3f0f-2d2c-4daf-bffc-7a9e84b82d86,76,start
104196,36fe3f0f-2d2c-4daf-bffc-7a9e84b82d86,77,win
104197,36fe3f0f-2d2c-4daf-bffc-7a9e84b82d86,77,start


In [137]:
#подсчет количества уникальных пользователей стартовавших уровень и всего количество стартов
t=df[df['action_type'] == 'start'].groupby(['completed_lvls']).agg({'user_id':['nunique','count']}).reset_index(col_fill='level')
t.columns = t.columns.droplevel(0)
t['perc_started_from_1']=(t['nunique']/t['nunique'].iloc[0]*100).round(2)
t['perc_started_from_prev'] = ((t['nunique'])/(t['nunique'].shift(1))*100).round(2)
t.rename(columns={"nunique": "users_started", "count": "starts"}, inplace=True)
t


,level,users_started,starts,perc_started_from_1,perc_started_from_prev
0,0,1666,1698,100.00,NaN
1,1,1517,1544,91.06,91.06
2,2,1392,1429,83.55,91.76
3,3,1309,1323,78.57,94.04
4,4,1240,1357,74.43,94.73
...,...,...,...,...,...
95,95,50,50,3.00,94.34
96,96,49,62,2.94,98.00
97,97,48,79,2.88,97.96
98,98,47,58,2.82,97.92


In [140]:
#количество побед на уровне
t_win=df[df['action_type'] == 'win'].groupby(['completed_lvls']).agg({'user_id':'count'}).reset_index()
t_win['completed_lvls_new']=t_win['completed_lvls']-1
t_win.rename(columns={"user_id": "wins"}, inplace=True)
t_win

,completed_lvls,wins,completed_lvls_new
0,1,1566,0
1,2,1466,1
2,3,1367,2
3,4,1284,3
4,5,1201,4
...,...,...,...
95,96,49,95
96,97,49,96
97,98,47,97
98,99,47,98


In [159]:
d = t.merge(t_win, left_on='level', right_on='completed_lvls_new')
d['winrate'] = (d['wins']/d['starts']).round(2)
d['level_new']=d['level']+1
d = d.drop(columns='level')
d = d.rename(columns={"level_new": "level"})
result = d[['level','users_started','starts','wins','perc_started_from_1','perc_started_from_prev','winrate']]
result


,level,users_started,starts,wins,perc_started_from_1,perc_started_from_prev,winrate
0,1,1666,1698,1566,100.00,NaN,0.92
1,2,1517,1544,1466,91.06,91.06,0.95
2,3,1392,1429,1367,83.55,91.76,0.96
3,4,1309,1323,1284,78.57,94.04,0.97
4,5,1240,1357,1201,74.43,94.73,0.89
...,...,...,...,...,...,...,...
95,96,50,50,49,3.00,94.34,0.98
96,97,49,62,49,2.94,98.00,0.79
97,98,48,79,47,2.88,97.96,0.59
98,99,47,58,47,2.82,97.92,0.81
